GitHub Repository Lab.docx

-- This cell counts the number of rows in the 'device_message_raw' and 'rapid_step_test_raw' tables in the workspace.bronze schema, displaying the results together.

In [0]:
%sql
SELECT 'device_message_raw' AS table_name, COUNT(*) AS rows FROM workspace.bronze.device_message_raw
UNION ALL
SELECT 'rapid_step_test_raw', COUNT(*) FROM workspace.bronze.rapid_step_test_raw;

This cell queries the `device_message_raw` table to find the top 10 devices by message count, grouping by `device_id` and ordering by the number of messages in descending order.

In [0]:
%sql
SELECT device_id, COUNT(*) AS messages
FROM workspace.bronze.device_message_raw
GROUP BY device_id
ORDER BY messages DESC
LIMIT 10;

This cell creates a temporary view `device_messages_clean` by cleaning and casting fields from the `device_message_raw` table, then displays the first 20 rows to verify the cleaning process.

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW device_messages_clean AS
SELECT
  device_id,
  TRY_CAST(REGEXP_REPLACE(CAST(distance AS STRING), '[^0-9.]', '') AS DOUBLE) AS distance_cm,
  CAST(timestamp AS BIGINT) AS ts_ms,
  CAST(date AS BIGINT) AS date_ms,
  message_origin,
  sensor_type,
  message
FROM workspace.bronze.device_message_raw;

-- Check the cleaning
SELECT * FROM device_messages_clean LIMIT 20

This cell calculates the average, minimum, and maximum cleaned distance values (`distance_cm`) for each device in the `device_messages_clean` view, displaying the top 10 devices with the highest average distance.

In [0]:
%sql
SELECT device_id,
AVG(distance_cm) AS avg_cm,
MIN(distance_cm) AS min_cm,
MAX(distance_cm) AS max_cm
FROM device_messages_clean
GROUP BY device_id
ORDER BY avg_cm DESC
LIMIT 10;

This cell analyzes step intervals from the `rapid_step_test_raw` table by exploding the `step_points` array, then calculates the total number of steps, average step interval, and standard deviation of step intervals for each customer and device, displaying the top 10 by step count.

In [0]:
%sql
-- One row per step interval
WITH exploded AS (
SELECT
customer,
device_id,
start_time,
posexplode(step_points) AS (step_index, step_ms)
FROM workspace.bronze.rapid_step_test_raw
)
SELECT customer, device_id,
COUNT(*) AS steps,
AVG(step_ms) AS avg_step_ms,
STDDEV(step_ms) AS sd_step_ms
FROM exploded
GROUP BY customer, device_id
ORDER BY steps DESC
LIMIT 10

This cell joins device messages with step test windows for each device, counting the number of distance readings and calculating average, minimum, and maximum cleaned distance values within each test window. The results are grouped by customer, device, and test window, showing the top 20 windows with the most readings.

In [0]:
%sql
-- Join messages that occurred during each test window for the same device
WITH tests AS (
SELECT customer, device_id, start_time, stop_time, test_time, total_steps
FROM workspace.bronze.rapid_step_test_raw
),
msgs AS (
SELECT device_id, ts_ms, distance_cm
FROM device_messages_clean
WHERE distance_cm IS NOT NULL
)
SELECT
t.customer, t.device_id, t.start_time, t.stop_time,
COUNT(m.ts_ms) AS readings_in_window,
AVG(m.distance_cm) AS avg_cm_in_window,
MIN(m.distance_cm) AS min_cm_in_window,
MAX(m.distance_cm) AS max_cm_in_window
FROM tests t
JOIN msgs m
ON m.device_id = t.device_id
AND m.ts_ms BETWEEN t.start_time AND t.stop_time
GROUP BY t.customer, t.device_id, t.start_time, t.stop_time
ORDER BY readings_in_window DESC
LIMIT 20;